In [41]:
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import re
import seaborn as sns
import matplotlib.pyplot as plt

/Users/dayosangowawa/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from pactools.grid_search import GridSearchCVProgressBar
from sklearn.metrics import  plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve, roc_auc_score, average_precision_score
from sklearn.preprocessing import label_binarize
import imblearn
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, classification_report
from sklearn import preprocessing
import scikitplot as skplt
from sklearn.feature_selection import SelectKBest, chi2

In [43]:
binarized_df = pd.read_csv('/Users/dayosangowawa/Desktop/GA/DSI20-lessons/projects/project-capstone/My Capstone/Forcibly displaced persons/cleaned_unhcrdf_final.csv')
binarized_df.head()

,Unnamed: 0,country_of_origin,country_of_asylum,population_type,urban_or_rural_location,accommodation_type,female_aged_0-4 years,female_aged_5-11 years,female_aged_12-17 years,female_aged_18-59 years,female_aged_over_60_years,male_aged_0-4_years,male_aged_5-11_years,male_aged_12-17 years,male_aged_18-59_years,male_aged_over_60_years
0,0,Colombia,Aruba,ASY,Urban,I,0,0,0,0,0,0,0,0,5,0
1,1,Cuba,Aruba,ASY,Urban,I,0,0,0,0,0,0,0,0,0,0
2,2,Afghanistan,Afghanistan,IDP,Rural,I,0,14,16,37,0,0,15,17,38,5
3,3,Afghanistan,Afghanistan,IDP,Urban,Q,74,810,853,2004,140,75,840,921,2278,206
4,4,Afghanistan,Afghanistan,IDP,Urban,Q,12,130,138,321,21,12,135,147,365,33


In [44]:
binarized_df.drop(columns = ['Unnamed: 0'], inplace=True)

/Users/dayosangowawa/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
binarized_df.head()

,country_of_origin,country_of_asylum,population_type,urban_or_rural_location,accommodation_type,female_aged_0-4 years,female_aged_5-11 years,female_aged_12-17 years,female_aged_18-59 years,female_aged_over_60_years,male_aged_0-4_years,male_aged_5-11_years,male_aged_12-17 years,male_aged_18-59_years,male_aged_over_60_years
0,Colombia,Aruba,ASY,Urban,I,0,0,0,0,0,0,0,0,5,0
1,Cuba,Aruba,ASY,Urban,I,0,0,0,0,0,0,0,0,0,0
2,Afghanistan,Afghanistan,IDP,Rural,I,0,14,16,37,0,0,15,17,38,5
3,Afghanistan,Afghanistan,IDP,Urban,Q,74,810,853,2004,140,75,840,921,2278,206
4,Afghanistan,Afghanistan,IDP,Urban,Q,12,130,138,321,21,12,135,147,365,33


In [47]:
y = binarized_df.pop('accommodation_type')
X = pd.get_dummies(binarized_df, drop_first=True)

In [48]:
X.shape

(91183, 407)

In [55]:
X_best = SelectKBest(chi2, k=20)

In [56]:
X_best.fit_transform(X, y)

array([[ 0,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  1],
       [ 0, 14, 16, ...,  0,  0,  0],
       ...,
       [ 5,  5,  0, ...,  0,  0,  1],
       [ 6, 10,  6, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  1]])

In [57]:
chi2_scores = pd.DataFrame(list(zip(X.columns, X_best.scores_, X_best.pvalues_)),
                          columns=['Feature', 'Score', 'p-value'])

/Users/dayosangowawa/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
chi2_scores.sort_values(by='p-value', ascending=True).head(20)


/Users/dayosangowawa/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Feature,Score,p-value
0,female_aged_0-4 years,8.404779e+05,0.000000e+00
389,country_of_asylum_United Kingdom of Great Brit...,1.846556e+03,0.000000e+00
383,country_of_asylum_Turkey,3.169932e+03,0.000000e+00
375,country_of_asylum_Switzerland,4.041575e+03,0.000000e+00
339,country_of_asylum_Norway,1.596377e+03,0.000000e+00
276,country_of_asylum_France,3.517391e+03,0.000000e+00
247,country_of_asylum_Canada,2.523835e+03,0.000000e+00
9,male_aged_over_60_years,2.954798e+04,0.000000e+00
406,urban_or_rural_location_Urban,2.041438e+03,0.000000e+00
7,male_aged_12-17 years,4.677225e+05,0.000000e+00


In [97]:
chi2_scores.sort_values(by='p-value', ascending=True).head(21).reset_index().drop(columns=['index', 'Score']).rename(columns ={'Feature': 'Top Features'})

/Users/dayosangowawa/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Top Features,p-value
0,female_aged_0-4 years,0.000000e+00
1,country_of_asylum_United Kingdom of Great Brit...,0.000000e+00
2,country_of_asylum_Turkey,0.000000e+00
3,country_of_asylum_Switzerland,0.000000e+00
4,country_of_asylum_Norway,0.000000e+00
5,country_of_asylum_France,0.000000e+00
6,country_of_asylum_Canada,0.000000e+00
7,male_aged_over_60_years,0.000000e+00
8,urban_or_rural_location_Urban,0.000000e+00
9,male_aged_12-17 years,0.000000e+00


In [93]:
chi2_scores.sort_values(by='Score', ascending=False).head(21).reset_index().drop(columns=['index']).rename(columns ={'Feature': 'Top Features'})

/Users/dayosangowawa/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Top Features,Score,p-value
0,female_aged_18-59 years,1.340193e+06,0.000000e+00
1,male_aged_5-11_years,1.159791e+06,0.000000e+00
2,female_aged_5-11 years,1.045774e+06,0.000000e+00
3,male_aged_0-4_years,9.661804e+05,0.000000e+00
4,female_aged_0-4 years,8.404779e+05,0.000000e+00
5,female_aged_12-17 years,5.286151e+05,0.000000e+00
6,male_aged_12-17 years,4.677225e+05,0.000000e+00
7,male_aged_18-59_years,4.303833e+05,0.000000e+00
8,female_aged_over_60_years,3.989373e+04,0.000000e+00
9,male_aged_over_60_years,2.954798e+04,0.000000e+00


In [68]:
top_feat = []

for i, z in zip(chi2_scores['Feature'], chi2_scores['p-value']):
    if z == 0:
        top_feat.append(i)

In [69]:
top_feat

['female_aged_0-4 years',
 'female_aged_5-11 years',
 'female_aged_12-17 years',
 'female_aged_18-59 years',
 'female_aged_over_60_years',
 'male_aged_0-4_years',
 'male_aged_5-11_years',
 'male_aged_12-17 years',
 'male_aged_18-59_years',
 'male_aged_over_60_years',
 'country_of_asylum_Canada',
 'country_of_asylum_France',
 'country_of_asylum_Norway',
 'country_of_asylum_Switzerland',
 'country_of_asylum_Turkey',
 'country_of_asylum_United Kingdom of Great Britain and Northern Ireland',
 'urban_or_rural_location_Urban']

In [72]:
top_feat_df = pd.DataFrame(top_feat, columns=['Top Features'])
top_feat_df

,Top Features
0,female_aged_0-4 years
1,female_aged_5-11 years
2,female_aged_12-17 years
3,female_aged_18-59 years
4,female_aged_over_60_years
5,male_aged_0-4_years
6,male_aged_5-11_years
7,male_aged_12-17 years
8,male_aged_18-59_years
9,male_aged_over_60_years


In [63]:
chi2_scores.sort_values(by='Score', ascending=False).head(20)

,Feature,Score,p-value
3,female_aged_18-59 years,1.340193e+06,0.000000e+00
6,male_aged_5-11_years,1.159791e+06,0.000000e+00
1,female_aged_5-11 years,1.045774e+06,0.000000e+00
5,male_aged_0-4_years,9.661804e+05,0.000000e+00
0,female_aged_0-4 years,8.404779e+05,0.000000e+00
2,female_aged_12-17 years,5.286151e+05,0.000000e+00
7,male_aged_12-17 years,4.677225e+05,0.000000e+00
8,male_aged_18-59_years,4.303833e+05,0.000000e+00
4,female_aged_over_60_years,3.989373e+04,0.000000e+00
9,male_aged_over_60_years,2.954798e+04,0.000000e+00


In [60]:
top_features = chi2_scores.sort_values(by='p-value', ascending=True).head(20)['Feature']
list(top100features)

/Users/dayosangowawa/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['female_aged_0-4 years',
 'country_of_asylum_United Kingdom of Great Britain and Northern Ireland',
 'country_of_asylum_Turkey',
 'country_of_asylum_Switzerland',
 'country_of_asylum_Norway',
 'country_of_asylum_France',
 'country_of_asylum_Canada',
 'male_aged_over_60_years',
 'urban_or_rural_location_Urban',
 'male_aged_12-17 years',
 'female_aged_over_60_years',
 'female_aged_18-59 years',
 'male_aged_18-59_years',
 'male_aged_0-4_years',
 'female_aged_12-17 years',
 'male_aged_5-11_years',
 'female_aged_5-11 years',
 'country_of_asylum_Bulgaria',
 'country_of_asylum_Austria',
 'country_of_asylum_Greece',
 'country_of_asylum_Egypt',
 'country_of_asylum_Cyprus',
 'country_of_asylum_Hungary',
 'country_of_asylum_Slovenia',
 'country_of_asylum_Ethiopia',
 'country_of_origin_Azerbaijan',
 'country_of_asylum_Sweden',
 'country_of_asylum_Sudan',
 'country_of_asylum_New Zealand',
 'country_of_asylum_Namibia',
 'country_of_asylum_Azerbaijan',
 'country_of_asylum_Belgium',
 'country_of_asyl

In [83]:
chi2_scores['p-value'].describe()

/Users/dayosangowawa/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


count    4.070000e+02
mean     1.191949e-01
std      2.326823e-01
min      0.000000e+00
25%      7.432761e-20
50%      1.786166e-05
75%      9.785800e-02
max      9.949874e-01
Name: p-value, dtype: float64